In [317]:
import heapq
import numpy as np
import pandas as pd

In [318]:
class MinimumHeap:
  """
  A class representing a minimum heap.

  Attributes:
    heap (list): The list representing the minimum heap.

  Methods:
    push(value): Pushes a value into the minimum heap.
    pop(): Removes and returns the minimum value from the heap.
    peek(): Returns the minimum value from the heap without removing it.
    size(): Returns the number of elements in the heap.
  """
  def __init__(self):
    self.heap = []

  def push(self, value):
    heapq.heappush(self.heap, value)

  def pop(self):
    return heapq.heappop(self.heap)

  def peek(self):
    return self.heap[0] if self.heap else None

  def size(self):
    return len(self.heap)

In [319]:
class Graph:
  def __init__(self, numVertices):
    self.numVertices = numVertices
    self.adjM = np.zeros((numVertices, numVertices), dtype=int)

  def pushEdge(self, source, destination, weight=None):
    self.adjM[source, destination] = weight
    self.adjM[destination, source] = weight

  def getNeighbors(self, vertex):
    row = self.adjM[vertex]
    indices = np.nonzero(row)
    weights = row[indices]
    return list(zip(indices[0], weights))

  def getEdges(self):
    """
    Returns a list of all edges in the graph.

    Returns:
      A list of tuples representing the edges in the graph.
    """
    indices = np.nonzero(self.adjM)
    edges = list(zip(indices[0], indices[1], self.adjM[indices]))
    return edges

  def __str__(self):
    return str(self.adjM)

In [320]:
def MST_Prim(graph):
  """
  Finds the minimum spanning tree of a graph using Prim's algorithm.

  Args:
    graph: The graph to find the minimum spanning tree of.

  Returns:
    A new graph instance representing the minimum spanning tree of the input
  """
  mst = Graph(graph.numVertices)
  visited = np.zeros(graph.numVertices, dtype=bool)
  heap = MinimumHeap()
  heap.push((0, 0, None))

  while heap.size() > 0:
    weight, vertex, parent = heap.pop()
    if visited[vertex]:
      continue
    visited[vertex] = True
    if parent is not None:
      mst.pushEdge(parent, vertex, weight)
    for neighbor, weight in graph.getNeighbors(vertex):
      if not visited[neighbor]:
        heap.push((weight, neighbor, vertex))
  return mst

In [321]:
def TreePreorderWalk(graph):
  """
  Performs a preorder walk of a tree.

  Args:
    graph: The graph to perform the preorder walk on.

  Returns:
    A list of vertices in the order they were visited.
  """
  visited = np.zeros(graph.numVertices, dtype=bool)
  stack = [0]
  walk = []

  while len(stack) > 0:
    vertex = stack.pop()
    if visited[vertex]:
      continue
    visited[vertex] = True
    walk.append(vertex)
    for neighbor, _ in graph.getNeighbors(vertex):
      if not visited[neighbor]:
        stack.append(neighbor)
  return walk

In [322]:
def approxTSP(graph):
  """
  Computes an approximate solution to the traveling salesman problem using
  minimum spanning trees.

  Args:
    graph: The graph to compute the approximate solution for.

  Returns:
    A list of vertices in the order they are visited.
  """
  mst = MST_Prim(graph)
  walk = TreePreorderWalk(mst)
  walk.append(0)
  return walk

In [323]:
def perfectMatching(graph, consideredVertices=None):
  """
  Computes a minimum weight perfect matching for a graph considering only the vertices in the consideredVertices list.

  Args:
    graph: The graph to compute the minimum weight perfect matching for.
    consideredVertices: A list of vertices to consider for the perfect matching.

  Returns:
    A list of edges representing the minimum weight perfect matching.
  """
  if consideredVertices is None:
    consideredVertices = range(graph.numVertices)
  edges = graph.getEdges()
  edges = [edge for edge in edges if edge[0] in consideredVertices and edge[1] in consideredVertices]
  edges.sort(key=lambda x: x[2])
  matching = []
  visited = np.zeros(graph.numVertices, dtype=bool)
  for edge in edges:
    if edge[0] in consideredVertices and edge[1] in consideredVertices and not visited[edge[0]] and not visited[edge[1]]:
      matching.append(edge)
      visited[edge[0]] = True
      visited[edge[1]] = True
  return matching

In [324]:
def eulerianTour(graph):
  """
  Computes an Eulerian tour for a graph.

  Args:
    graph: The graph to compute the Eulerian tour for.

  Returns:
    A list of vertices in the order they are visited.
  """
  start_vertex = 0  # Choose a starting vertex
  visited = np.zeros(graph.numVertices, dtype=bool)
  tour = []
  stack = np.array([start_vertex])

  while stack.size > 0:
    vertex = stack[-1]
    if not visited[vertex]:
      visited[vertex] = True
      tour.append(vertex)
    neighbors = graph.getNeighbors(vertex)
    unvisited_neighbors = np.array([neighbor for neighbor, _ in neighbors if not visited[neighbor]])
    if unvisited_neighbors.size > 0:
      next_vertex = unvisited_neighbors[0]
      stack = np.append(stack, next_vertex)
    else:
      stack = stack[:-1]

  return tour

In [325]:
def christofidesAlgorithm(graph):
  """
  Computes an approximate solution to the traveling salesman problem using
  Christofides algorithm.

  Args:
    graph: The graph to compute the approximate solution for.

  Returns:
    A list of vertices in the order they are visited.
  """
  def eliminateDuplicated(tour):
    for vertex in tour:
      if tour.count(vertex) > 1:
        tour.remove(vertex)

  mst = MST_Prim(graph)
  oddDegreeVertices = []
  for vertex in range(mst.numVertices):
    if len(mst.getNeighbors(vertex)) % 2 == 1:
      oddDegreeVertices.append(vertex)
  matching = perfectMatching(graph, oddDegreeVertices)
  for edge in matching:
    mst.pushEdge(edge[0], edge[1], edge[2])
  tour = eulerianTour(mst)
  tour.append(0)
  return tour
  

In [326]:
# Class 13 slide problem
graph = Graph(5)
graph.pushEdge(0, 1, 4)
graph.pushEdge(0, 2, 8)
graph.pushEdge(0, 3, 9)
graph.pushEdge(0, 4, 12)
graph.pushEdge(1, 2, 6)
graph.pushEdge(1, 3, 8)
graph.pushEdge(1, 4, 9)
graph.pushEdge(2, 3, 10)
graph.pushEdge(2, 4, 11)
graph.pushEdge(3, 4, 7)

def getTourWeight(graph, tour):
  weight = 0
  for i in range(len(tour) - 1):
    weight += graph.adjM[tour[i], tour[i + 1]]
  return weight

approxTSPTour = approxTSP(graph)
print('Approximate TSP Algorithm')
print('Path:', approxTSPTour)
print('Total weight:', getTourWeight(graph, approxTSPTour))

print('\n')

christofidesTour = christofidesAlgorithm(graph)
print('Christofides Algorithm')
print('Path:', christofidesTour)
print('Total weight:', getTourWeight(graph, christofidesTour))

Approximate TSP Algorithm
Path: [0, 1, 3, 4, 2, 0]
Total weight: 38


Christofides Algorithm
Path: [0, 1, 2, 4, 3, 0]
Total weight: 37
